# Make odgt files 


## Make odgt files from clusters

In [3]:
def odgt_format_from_dir(filename, labeled_img_dir, all_img_dir, labels_dir, dest_filename, extension):
    df_seqs = pd.read_csv(filename , sep=":", names=['img', 'label'])
    with open(dest_filename + ".odgt", "w") as fw:

        for img in glob.glob(labeled_img_dir + '/*.jpg'):
            img_name = img.split("/")[-1]
            img_path = os.path.join(all_img_dir, img_name)
            w, h = cv2.imread(img_path).shape[:2]
            cluster_name = df_seqs[df_seqs['img'].str.contains(img_name)].reset_index(drop=True)['label'][0]
            seq_match = df_seqs[df_seqs['label'] == cluster_name].reset_index(drop=True)
            if seq_match.shape[0] >= 2:
                img_names = [img_path]
                seg = labels_dir + img_name.split('.')[0] + '.png'
                seg_names = [seg] * 2
                for row in seq_match.iterrows():
                    if row[1].img not in img_names:
                        img_names.append(row[1].img)
                        w, h = cv2.imread(row[1].img).shape[:2]
                    if len(img_names) == 2:
                        new_line = {}
                        new_line["fpath_img"], new_line["fpath_segm"], new_line["width"], new_line["height"] = \
                                img_names, seg_names, w, h
                        json.dump(new_line, fw)
                        fw.write('\n')
                        img_names = [img_path]
                        seg_names = [seg] * 2
def sup_odgt_format( img_dir, label_dir, dest_filename, extension):
    with open(dest_filename + ".odgt", "w") as fw:
        for path in glob.glob(img_dir + "*." + extension):
            w, h = cv2.imread(path).shape[:2]
            img_name = path.split("/")[-1]
            #print(img_name)
            img_names = img_dir + img_name
            seg_names = label_dir + img_name.split('.')[0] + '.png'
            new_line = {}
            new_line["fpath_img"], new_line["fpath_segm"], new_line["width"], new_line["height"] = \
                    img_names, seg_names, w, h
            json.dump(new_line, fw)
            fw.write('\n')

 ### Get odgts for seqs, supervised, and val data

In [4]:
cluster_files = ['VOCallMinusVal1000FCM', 'VOCallMinusVal1000', 'VOCallMinusVal4096', 'VOCallMinusValFCM']
dest_files = ['1000FCM', '1000', '4096', 'FCM']
root_dir = "/data/sara/DecompositionFeatureSegmentation/"
for ind, _file in enumerate(cluster_files):
    ## only to get the pairs
    cluster_file = "/data/sara/DecompositionFeatureSegmentation/clustering/" + _file
    labeled_img_dir = "/data/sara/semantic-segmentation-pytorch/datasets/VOC/train_imgs/"    
    all_img_dir = "/data/sara/semantic-segmentation-pytorch/datasets/VOC/train_and_unlabeled_imgs/VOCallMinusVal/"
    labels_dir = "/data/sara/semantic-segmentation-pytorch/datasets/VOC/train_labels/"
    odgt_format_from_dir(cluster_file, labeled_img_dir,all_img_dir ,labels_dir, root_dir+"seqs_odgts/VOCpairs_"+dest_files[ind], 'jpg')
    
    
    ## only to get the train labeled ones
    img_dir = "/data/sara/semantic-segmentation-pytorch/datasets/VOC/train_imgs/"
    label_dir = "/data/sara/semantic-segmentation-pytorch/datasets/VOC/train_labels/"
    sup_odgt_format(img_dir, label_dir, root_dir+"supervised_odgts/VOCtrain_"+dest_files[ind], 'jpg')
    
    ## only to get the val labeled ones
    img_dir = "/data/sara/semantic-segmentation-pytorch/datasets/VOC/val_imgs/"
    label_dir = "/data/sara/semantic-segmentation-pytorch/datasets/VOC/val_labels/"
    sup_odgt_format(img_dir, label_dir, root_dir+"val_odgts/VOCval_"+dest_files[ind], 'jpg')